In [55]:
import pandas as pd
import numpy as np
from scipy.integrate import simps
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations, groupby
import networkx as nx
from torch.fx.experimental.unification.unification_tools import groupby

In [2]:
df = pd.read_csv('/home/eduardoferreira/PycharmProjects/Rec_Sys/dataset/ml-latest-small/ratings.csv', sep=',')

In [3]:
matrix = df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

In [4]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [7]:
def find_n_rated(df1, df2):
    mask = (df1 > 0) & (df2 > 0)
    # df = pd.merge(df1,df2, on='movieId', how ='inner')
    if df1[mask].shape[0] > 10:
        return df1[mask].values, df2[mask].values
    else:
        return None

In [9]:
# similarity_list = []
# n = matrix.shape[0]
# # n = 50
# for i in range(n):
#     j = i + 1
#     while j < n - 1:
#         if find_n_rated(matrix.iloc[i], matrix.iloc[j + 1]) is not None:
#             v1, v2 = find_n_rated(matrix.iloc[i], matrix.iloc[j + 1])
#             dist = cos_sim(v1, v2)
#             similarity_list.append({'userId1': i, 'userId2': j, 'similarity': dist})
#         j = j + 1


In [10]:
# def similarity_users(df: pd.DataFrame):
#     for i in range(df.shape[0]):


In [11]:
def calculate_user_similarity(df, n):
    ratings_matrix = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

    # Generate pairs of user IDs
    user_pairs = list(combinations(ratings_matrix.index, 2))

    similarity_data = []
    for (u1, u2) in user_pairs:
        # Find common movies rated by both users
        common_movies = ratings_matrix.loc[u1].index.intersection(ratings_matrix.loc[u2].index)

        # Continue if there are not enough common movies rated by both users
        if len(common_movies) <= n:
            continue

        # Calculate cosine similarity using ratings of common movies
        ratings_u1 = ratings_matrix.loc[u1, common_movies].values.reshape(1, -1)
        ratings_u2 = ratings_matrix.loc[u2, common_movies].values.reshape(1, -1)
        similarity = cosine_similarity(ratings_u1, ratings_u2)[0, 0]

        # Append to similarity_data
        similarity_data.append((u1, u2, similarity))

    # Create DataFrame with columns 'userId', 'userId1', and 'similarity'
    similarity_df = pd.DataFrame(similarity_data, columns=['userId', 'userId1', 'similarity'])

    return similarity_df


similarity_df = calculate_user_similarity(df, n=10)
print(similarity_df)

In [84]:
similarity_df.sort_values(by='similarity', ascending=False, inplace=True)
similarity_df.to_csv('/home/eduardoferreira/PycharmProjects/Rec_Sys/dataset/treated/similarity_users.csv', sep=',')

In [56]:
def group_users_by_similarity(df, n_groups):
    X = df[['userId', 'userId1', 'similarity']]

    kmeans = KMeans(n_clusters=n_groups, random_state=42)
    X['group'] = kmeans.fit_predict(X[['similarity']])

    X['user'] = X[['userId', 'userId1']].values.tolist()
    X = X.drop(['userId', 'userId1'], axis=1)

    grouped_df = X.groupby('group')['user'].apply(list).reset_index()

    return grouped_df


grouped_df = group_users_by_similarity(similarity_df, n_groups=6)

In [177]:
similarity_df[['userId', 'userId1']].astype(int)

In [184]:
def group_users(similarity: pd.DataFrame, n_groups) -> []:
    group = []

    for g in range(n_groups):
        for i in range(similarity.shape[0]):
            # print('group')
            if not group:
                group.append([g, similarity.iloc[i]['userId'], similarity.iloc[i]['userId1']])
            else:
                if similarity.iloc[i]['userId'] in group or similarity.iloc[i]['userId1'].all() in group:
                    pass
    return group

In [216]:
# test = group_users(similarity_df, 1)

In [232]:
 [similarity_df[similarity_df['similarity'] > 0.5]['userId'] == 126][0]


In [212]:
sim = similarity_df.groupby(['userId'])

In [213]:
sim.head()